<a href="https://colab.research.google.com/github/MCFreddie777/nsiete-parkinson/blob/master/script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
!pip install wandb &> /dev/null

In [73]:
import pandas as pd
import wandb
from wandb import AlertLevel
from datetime import timedelta

In [74]:
wandb.login()

True

In [75]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

Ako prvý krok načítame dáta a pozrieme sa na vzorku atribútov.

In [76]:
data_path = 'https://raw.githubusercontent.com/MCFreddie777/nsiete-parkinson/master/data/' if IN_COLAB else 'data/' # Read files directly from github if working in google colab
data_file = 'parkinsons.data'
data = pd.read_csv(f"{data_path}{data_file}")

In [77]:
data.head()

,name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,MDVP:APQ,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,phon_R01_S01_1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,0.426,0.02182,0.03130,0.02971,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,phon_R01_S01_2,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,0.626,0.03134,0.04518,0.04368,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,phon_R01_S01_3,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,0.482,0.02757,0.03858,0.03590,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,phon_R01_S01_4,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,0.517,0.02924,0.04005,0.03772,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,phon_R01_S01_5,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,0.584,0.03490,0.04825,0.04465,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


Atribúty:

- **name** - ASCII subject name and recording number
- **MDVP:Fo(Hz)** - Average vocal fundamental frequency
- **MDVP:Fhi(Hz)** - Maximum vocal fundamental frequency
- **MDVP:Flo(Hz)** - Minimum vocal fundamental frequency
- **MDVP:Jitter(%), MDVP:Jitter(Abs), MDVP:RAP, MDVP:PPQ, Jitter:DDP** - Several measures of variation in fundamental frequency
- **MDVP:Shimmer, MDVP:Shimmer(dB), Shimmer:APQ3, Shimmer:APQ5, MDVP:APQ, Shimmer:DDA** - Several measures of variation in amplitude 
- **NHR,HNR** - Two measures of ratio of noise to tonal components in the voice
- **status** - Health status of the subject (one) - Parkinson's, (zero) - healthy
- **RPDE, D2** - Two nonlinear dynamical complexity measures
- **DFA** - Signal fractal scaling exponent
- **spread1, spread2, PPE** - Three nonlinear measures of fundamental frequency variation


Pozrieme sa aký počet dát máme:

In [78]:
len(data)

195

## Tensorflow / Keras

In [79]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import numpy as np
from matplotlib import pyplot as plt

from wandb.keras import WandbCallback

Najprv si dáta rozdelime na trenovaciu a testovaciu sadu v pomere 70 k 30.

In [80]:
X_data = data.loc[:, data.columns != 'status'].drop(['name'], axis=1)
Y_data = data['status']

In [81]:
X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=0.3, random_state=256)

Nastavíme si parametre modelu - ako napríklad počet epôch, learning rate...

In [82]:
#MODEL PARAMETERS 
config = {
"EPOCHS": 100,
"BATCH_SIZE": 195,
"LEARNING_RATE": 0.0001,
"MOMENTUM": 0.5,
"epsilon": 1e-07,
"rho":0.9
}

Zadefinujeme si model - 3 skryté vrstvy s RELU aktivačnou funkciou. Na výstupe je 1 neurón s aktivačnou funkciou sigmoid, nakolko ide o binárnu klasifikáciu atribútu 'status'.

In [83]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(22,)),
    keras.layers.Dense(22, activation=tf.nn.relu),
	  keras.layers.Dense(32, activation=tf.nn.relu),
	  keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid),
])

Nastavíme modelu parametre - loss funkciu, optimizer...

In [84]:
model.compile(
    optimizer=keras.optimizers.RMSprop(
        learning_rate=config["LEARNING_RATE"], 
        rho=config["rho"], 
        momentum=config["MOMENTUM"],
        epsilon=config["epsilon"]
        ),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [85]:
run = wandb.init(project="parkinson_tensor", entity="nn2021")

In [86]:
wandb.config.update(config)

Po nastavení wandb spustíme trénovanie siete.

In [87]:
model.fit(X_train, y_train, epochs=config["EPOCHS"], batch_size=config["BATCH_SIZE"], validation_data=(X_test, y_test), callbacks=[WandbCallback()])

Epoch 1/100
1/1 [==============================] - 1s 744ms/step - loss: 8.4138 - accuracy: 0.7132 - val_loss: 4.7619 - val_accuracy: 0.8475
Epoch 2/100
1/1 [==============================] - 0s 50ms/step - loss: 8.1409 - accuracy: 0.7132 - val_loss: 4.5713 - val_accuracy: 0.8475
Epoch 3/100
1/1 [==============================] - 0s 55ms/step - loss: 7.8084 - accuracy: 0.7132 - val_loss: 4.3828 - val_accuracy: 0.8475
Epoch 4/100
1/1 [==============================] - 0s 59ms/step - loss: 7.4780 - accuracy: 0.7132 - val_loss: 4.2064 - val_accuracy: 0.8475
Epoch 5/100
1/1 [==============================] - 0s 60ms/step - loss: 7.1646 - accuracy: 0.7132 - val_loss: 4.0427 - val_accuracy: 0.8475
Epoch 6/100
1/1 [==============================] - 0s 57ms/step - loss: 6.8717 - accuracy: 0.7132 - val_loss: 3.8909 - val_accuracy: 0.8475
Epoch 7/100
1/1 [==============================] - 0s 53ms/step - loss: 6.5923 - accuracy: 0.7132 - val_loss: 3.7434 - val_accuracy: 0.8475
Epoch 8/100
1/1 [==

Výsledky nahráme do wandb.

In [88]:
run.finish()

epoch,99
loss,0.55835
accuracy,0.77206
val_loss,0.54239
val_accuracy,0.89831
_runtime,12
_timestamp,1616883406
_step,99
best_val_loss,0.45589
best_epoch,42


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▇▆▅▅▄▄▄▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▁▁▅▄▃▄▄▄▄▅▅▅▅▅▅▅▅▅▅▆▇▇▇▆█
val_loss,█▇▇▆▅▅▄▄▄▃▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆█████▆█
_runtime,▁▁▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇█████
_timestamp,▁▁▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇█████
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


## Pytorch

In [89]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from itertools import chain

from sklearn.preprocessing import StandardScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

Opäť, rozdelíme si set na dáta a target (triedu).

In [90]:
X_data = data.loc[:, data.columns != 'status'].drop(['name'], axis=1)
Y_data = data['status']
format(X_data.shape)

'(195, 22)'

Rozdelenie dát na testovaciu a trénovaciu sadu.

In [91]:
X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=0.3, random_state=256)

Pripravíme dáta na prácu s PyTorchom

In [92]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [93]:
class trainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

class testData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)

In [94]:
train_data = trainData(torch.FloatTensor(X_train),torch.FloatTensor(y_train))
test_data = testData(torch.FloatTensor(X_test))

Nastavíme parametre pre model

In [95]:
#MODEL PARAMETERS 
config = {
"EPOCHS": 50,
"BATCH_SIZE": 195,
"LEARNING_RATE": 0.0001,
"MOMENTUM": 0.9,
"epsilon": 1e-07,
"alpha":0.9,
"DROPOUT": 0.1}

In [96]:
train_loader = DataLoader(dataset=train_data, batch_size=config["BATCH_SIZE"] , shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=config["BATCH_SIZE"], shuffle=True)

Zadefinujeme si model s jednou skrytou vrstvou. Skúšali sme viacej vrstiev, ale znižovalo nám to presnosť. Použili sme  aktivačnú funkciu RELU.

In [97]:
class binaryClassification(nn.Module):
    def __init__(self):
        super(binaryClassification, self).__init__()
    
        self.layer_1 = nn.Linear(22, 64) 
        self.layer_2 = nn.Linear(64, 32)
        self.layer_out = nn.Linear(32, 1) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=config["DROPOUT"])

        
    def forward(self, inputs):
        x = self.layer_1(inputs)
        x = self.relu(x)
        x = self.layer_2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        
        return x

In [98]:
model = binaryClassification()
model.to('cpu')

binaryClassification(
  (layer_1): Linear(in_features=22, out_features=64, bias=True)
  (layer_2): Linear(in_features=64, out_features=32, bias=True)
  (layer_out): Linear(in_features=32, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
)

In [99]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.RMSprop(model.parameters(), lr=config["LEARNING_RATE"], alpha=config["alpha"], eps=config["epsilon"], momentum=config["MOMENTUM"])

In [100]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum / y_test.shape[0]
        
    return acc

In [101]:
run = wandb.init(project="parkinson_pytorch", entity="nn2021")

In [102]:
wandb.config.update(config)

In [103]:
wandb.watch(model)

Po inicializácii wandb spustíme trénovanie siete v jednotlivých epochách po batchoch.

In [104]:
model.train()

epoch_loss_arr = []
epoch_acc_arr = []

for e in range(1, config["EPOCHS"]+1):

    epoch_loss = 0
    epoch_acc = 0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to("cpu"), y_batch.to("cpu")
        optimizer.zero_grad()
        

        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()

    epoch_loss_arr.append(epoch_loss / len(train_loader))
    epoch_acc_arr.append(epoch_acc / len(train_loader))
    
    result_loss = epoch_loss / len(train_loader)
    result_acc = epoch_acc / len(train_loader)

    wandb.log({"loss train" : loss})
    wandb.log({"accurancy train" : result_acc})

    
    y_pred_list = []
    model.eval()

    with torch.no_grad():
        for X_batch in test_loader:
            X_batch = X_batch.to("cpu")
            y_test_pred = model(X_batch)
            y_test_pred = torch.sigmoid(y_test_pred)
            y_pred_tag = torch.round(y_test_pred)
            y_pred_list.append(y_pred_tag.cpu().numpy())

    y_pred_list = torch.FloatTensor(list(chain.from_iterable(list(chain.from_iterable(y_pred_list)))))
    y_test_tensor = torch.FloatTensor(y_test.to_list())
      
    wandb.log({"loss test": criterion(y_test_tensor,y_pred_list )})
    wandb.log({"accurancy test": binary_acc(y_test_tensor,y_pred_list )})

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


In [105]:
run.finish()

loss train,0.35099
_runtime,4
_timestamp,1616883414
_step,199
accurancy train,0.82353
loss test,0.65935
accurancy test,0.61017


loss train,█████▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁
_runtime,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁████████
_timestamp,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁████████
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
accurancy train,▁▁▂▃▆▆▆▆▇████████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█
loss test,▄▃▃▃▁▁▁▁▂▂▃▃▃▃▄▅▅▆▅▆▇▆▆▇▆▇▆▇▆▆▇▆▇▇▇███▇▇
accurancy test,▄▆▆▆█▇▇▇▇█▆▆▆▇▆▄▄▃▅▄▁▄▃▁▃▂▃▂▄▄▃▃▂▃▂▂▁▂▃▃
